Naive retrieval without reranking

In [1]:
# simple hack to support import module from parent directory
import sys
sys.path.append('../')

from rag_llama.core.retrievers import StandardRetriever

In [2]:
knowledge_base = '../data/Tesla_manual_embeddings.pk'
standard_retriever = StandardRetriever(knowledge_embed=knowledge_base)

model_ckpt_dir = '../checkpoints/finetune_embedding/epoch-50'
tokenizer_ckpt_dir = '../checkpoints/finetune_embedding/tokenizer'
finetuned_knowledge_base = '../data/Tesla_manual_embeddings_finetuned.pk'
finetuned_retriever = StandardRetriever(knowledge_embed=finetuned_knowledge_base, model_ckpt_dir=model_ckpt_dir, tokenizer_ckpt_dir=tokenizer_ckpt_dir)


Loading sentence-transformers/all-MiniLM-L6-v2 tokenizer from HuggingFace...
Loading sentence-transformers/all-MiniLM-L6-v2 model from HuggingFace...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading tokenizer from ../checkpoints/finetune_embedding/tokenizer...
Loading model from ../checkpoints/finetune_embedding/epoch-50...


In [3]:
from IPython.display import HTML, display

def display_in_columns(query: str, standard_result: str, finetuned_result: str) -> None:
    """
    Display content in columns side-by-side.
    
    Parameters:
    query (str): user query
    standard_result (str): response for left column.
    finetuned_result (str): response for right column.

    """

    # Convert newlines to HTML line breaks and bulleted lists to HTML list items
    def handle_newline(input_strings) -> str:
        if input_strings is None:
            return
        return input_strings.replace('\n', '<br>')
    
    standard_result = handle_newline(standard_result)
    finetuned_result = handle_newline(finetuned_result)

    html_content = f'''
    <div>
        <div style="padding: 20px 0; font-weight: bold;">User: {query}</div>
        <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px;">
            <div>
                <div style="font-weight: bold;">Standard Retrieval:</div>
                {standard_result}
            </div>

            <div>
                <div style="font-weight: bold;">Retrieval with fine-tuned embedding:</div>
                {finetuned_result}
            </div>
        </div>
    <div>
    '''
    display(HTML(html_content))


def format_document_str(data):

    formatted_text = f"Document: {data['document_title']}\nCar model: {data['car_model']}\nSubject: {data['subject']} - {data['section']}\nContent: {data['content']}\nPage: {data['page']}"
    return formatted_text


def find_top_matches(question: str, top_k=1):
    standard_top_matches = standard_retriever.retrieve(question, top_k)
    finetuned_top_matches = finetuned_retriever.retrieve(question, top_k)

    standard_result = format_document_str(standard_top_matches[0])
    finetuned_result = format_document_str(finetuned_top_matches[0])

    display_in_columns(question, standard_result, finetuned_result)



The following alert codes are in the training datasets

In [4]:
question = "what does CP_a055 mean?"
find_top_matches(question)

In [5]:
question = "I see UMC_a008 on my dashboard what should I do"
find_top_matches(question)

In [6]:
question = "Why do I see code APP_w224 on the screen"
find_top_matches(question)

In [7]:
question = "How to fix error BMS_a068 on my car"
find_top_matches(question)

However, the following alert codes are only in the validation dataset, never seen by the model during training.

In [8]:
question = "what does BMS_a067 mean?"
find_top_matches(question)

In [9]:
question = "I see DI_a175 on my dashboard what should I do?"
find_top_matches(question)

In [10]:
question = "How to fix error CC_a005 on the screen"
find_top_matches(question)

## Test the fine-tuned model should still perform well on general questions

In [11]:
question = "How can I use a key card to open my Tesla?"
find_top_matches(question)